In [1]:
# -*- coding: utf-8 -*-
"""
泡泡玛特(HK:9992) 量化分析作业 - 2025年数据
日期：2025-07-20
"""

import numpy as np
import pandas as pd
import yfinance as yf
from scipy.stats import norm
import warnings
warnings.filterwarnings("ignore")  # 忽略警告信息

# ===== 数据获取 =====
ticker = "9992.HK"  # 泡泡玛特港股代码
start_date_2025 = "2025-01-01"
end_date_2025 = "2025-06-30"

print("正在尝试下载2025年数据...")
try:
    # 尝试下载2025年数据
    data_2025 = yf.download(ticker, start=start_date_2025, end=end_date_2025, progress=False)
    
    if data_2025.empty:
        raise ValueError("2025年数据为空，使用模拟数据替代")
    
    print("成功获取2025年数据!")
    closing_prices = data_2025['Close']
    data_source = "实际2025年数据"

except Exception as e:
    print(f"数据下载出错或为空: {e}")
    print("使用模拟数据替代...")
    
    # 创建模拟数据
    dates = pd.date_range(start_date_2025, end_date_2025, freq='B')  # 工作日
    np.random.seed(42)
    base_price = 30.0
    rand_returns = np.random.normal(0.001, 0.02, len(dates))
    prices = base_price * np.exp(np.cumsum(rand_returns))
    closing_prices = pd.Series(prices, index=dates)
    data_source = "模拟数据"

# 确保我们有一个有效的收盘价序列
if len(closing_prices) == 0:
    closing_prices = pd.Series([30.0], index=[pd.Timestamp(end_date_2025)])

# ===== Part A：收益与波动率分析 =====
if len(closing_prices) > 1:
    daily_returns = np.log(closing_prices / closing_prices.shift(1)).dropna()
    avg_daily_return = float(daily_returns.mean())
    daily_volatility = float(daily_returns.std())
else:
    avg_daily_return = 0.001
    daily_volatility = 0.02

annualized_return = float(avg_daily_return * 252)
annualized_volatility = float(daily_volatility * np.sqrt(252))

# ===== Part B：期权定价 =====
last_close = float(closing_prices.iloc[-1])
strike_price = round(last_close)
print(f"\n行权价选择: {strike_price} (接近期末收盘价 {last_close:.2f})")
print(f"数据来源: {data_source}")

S0 = last_close
K = strike_price
T = 30/365
r = 0.04
sigma = annualized_volatility

d1 = (np.log(S0 / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
d2 = d1 - sigma * np.sqrt(T)

call_price = S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
put_price = K * np.exp(-r * T) * norm.cdf(-d2) - S0 * norm.cdf(-d1)

# ===== Part C：看跌-看涨平价验证 =====
left_side = call_price - put_price
right_side = S0 - K * np.exp(-r * T)
deviation = abs(left_side - right_side)

# ===== 结果输出 =====
print("\n=== Part A 结果 ===")
print(f"日均收益率: {avg_daily_return:.6f}")
print(f"日波动率: {daily_volatility:.6f}")
print(f"年化收益率: {annualized_return:.6f}")
print(f"年化波动率: {annualized_volatility:.6f}")

print("\n=== Part B 结果 ===")
print(f"当前日期: 2025-07-01 (假设)")
print(f"期权到期日: 2025-07-31")
print(f"无风险利率: {r * 100:.1f}%")
print(f"看涨期权价格: {call_price:.4f}")
print(f"看跌期权价格: {put_price:.4f}")

print("\n=== Part C 结果 ===")
print(f"左式 (C-P): {left_side:.4f}")
print(f"右式 (S-Ke^(-rT)): {right_side:.4f}")
print(f"偏差: {deviation:.8f}")
print("验证结论: " + ("通过" if deviation < 1e-5 else "不通过"))
print("偏差可能原因: 交易成本、股息、市场摩擦或四舍五入误差")

print("\n数据预览(2025年):")
print(closing_prices.tail())

# 保存结果到CSV（可选）
results = {
    "股票代码": ticker,
    "分析期间": f"{start_date_2025} 至 {end_date_2025}",
    "数据来源": data_source,
    "日均收益率": avg_daily_return,
    "日波动率": daily_volatility,
    "年化收益率": annualized_return,
    "年化波动率": annualized_volatility,
    "当前股价": last_close,
    "行权价": strike_price,
    "看涨期权价格": call_price,
    "看跌期权价格": put_price,
    "平价验证偏差": deviation
}
pd.DataFrame([results]).to_csv("泡泡玛特_量化分析结果.csv", index=False)

正在尝试下载2025年数据...
成功获取2025年数据!

行权价选择: 258 (接近期末收盘价 257.80)
数据来源: 实际2025年数据

=== Part A 结果 ===
日均收益率: 0.008846
日波动率: 0.043851
年化收益率: 2.229103
年化波动率: 0.696111

=== Part B 结果 ===
当前日期: 2025-07-01 (假设)
期权到期日: 2025-07-31
无风险利率: 4.0%
看涨期权价格: 20.7904
看跌期权价格: 20.1436

=== Part C 结果 ===
左式 (C-P): 0.6468
右式 (S-Ke^(-rT)): 0.6468
偏差: 0.00000000
验证结论: 通过
偏差可能原因: 交易成本、股息、市场摩擦或四舍五入误差

数据预览(2025年):
Ticker         9992.HK
Date                  
2025-06-23  244.199997
2025-06-24  252.199997
2025-06-25  247.399994
2025-06-26  253.000000
2025-06-27  257.799988
